This notebook reads info from a given catalog and saves the specified waveform traces as SAC files. Original author: Axel Wang/Xiaohan Song. Significant modifications by Emmanuel, though the unefficient cata structure has not yet been fixed.

<span style="color:red; font-weight:bold;">CHANGE THINGS IN 2 AREAS:</span><br>
<span style="color:red; font-weight:bold;">1. File/directory naming under Setup</span><br>
<span style="color:red; font-weight:bold;">2. Study area and station under Process --> Download info</span>

# Import

In [1]:
from obspy import read, read_inventory
from obspy.geodetics import degrees2kilometers
from obspy.signal import rotate
import math
from scipy.io import savemat
from os.path import join as pjoin
from obspy import Trace
from obspy import Stream
import numpy as np
import glob
from obspy.signal import filter
from obspy.clients.fdsn import Client
from obspy.core.event import Origin
from obspy.geodetics import gps2dist_azimuth, kilometer2degrees
from obspy.taup import TauPyModel
from obspy.io.sac.sactrace import SACTrace
from obspy import read_inventory, UTCDateTime
from obspy import read
from obspy.signal.tf_misfit import plot_tfr
from obspy.signal.cross_correlation import correlate
import numpy as np
import matplotlib.pyplot as plt
from obspy.geodetics.base import gps2dist_azimuth
import csv
import os
import pathlib
from matplotlib.path import Path
import sympy as sp
from scipy.optimize import curve_fit
from scipy import interpolate

# Setup

In [2]:
# TOGGLE
name = 'WT_Final.txt'
direct = 'WT_Raw_Data/'

In [3]:
# Catalog to read
catalog = 'PROVIDED.cata'

# Client
client = Client("IRIS")
# Earth model
# model = TauPyModel(model="iasp91")
model = TauPyModel(model='/Users/emmanuelzheng/Downloads/Synthetics/Seisyn/Tibetan_model.npz')

# Folder to store data
directory = '/Users/emmanuelzheng/Downloads/Synthetics/Seiobs/'
# Directory
raw_directory = directory+direct # Directory to download raw data
# Create directory if it doesn't exist
os.makedirs(raw_directory, exist_ok=True)

# Ensure directory is empty
paths = pathlib.Path(raw_directory).glob('*.SAC') # Delete all existing SAC files in raw data directory
for path in paths:
    path.unlink()  # Remove each SAC file 

# Process

## Prep cata

In [4]:
# Initialize a 1x1 matrix to store earthquake catalog data
cata = np.mat([1])

# Open the catalog file 'PROVIDED.cata' for reading
with open(catalog) as fc:
    # Read all lines from the file into a list
    reader = fc.readlines()
    
    # Loop through each line in the catalog file
    for row in reader:
        # Split the line into individual values (space-separated)
        lst = row.split()
        
        # Line has fewer than 8 columns (incomplete data)
        if len(lst) < 8:
            # Check if this is the first valid entry
            if len(cata.transpose()) == 1:
                # Create first row of catalog with earthquake data
                # lst[0]+'T'+lst[1]: Combine date and time into ISO format
                # lst[3]: longitude
                # lst[4]: latitude  
                # (float(lst[5])+float(lst[5]))/2: depth (duplicated value divided by 2)
                # lst[2]: magnitude
                # lst[6]: depth error
                # lst[5]: depth again
                # lst[-1]: star (if exist) or random
                cata = np.mat([[lst[0]+'T'+lst[1], float(lst[3]), float(lst[4]), float(lst[5]), float(lst[2]), float(lst[6]), float(lst[5]), lst[-1]]])
            else:
                # Append new row to existing catalog matrix
                cata = np.r_[cata, np.mat([[lst[0]+'T'+lst[1], float(lst[3]), float(lst[4]), float(lst[5]), float(lst[2]), float(lst[6]), float(lst[5]), lst[-1]]])]
        
        # Line has 8 or more columns (complete data)
        else:
            # Check if this is the first valid entry
            if len(cata.transpose()) == 1:
                # Create first row with complete data including station info (last column contains station information)
                cata = np.mat([[lst[0]+'T'+lst[1], float(lst[3]), float(lst[4]), float(lst[5]), float(lst[2]), float(lst[6]), float(lst[5]), lst[-1]]])
            else:
                # Append new row with complete data to existing catalog
                cata = np.r_[cata, np.mat([[lst[0]+'T'+lst[1], float(lst[3]), float(lst[4]), float(lst[5]), float(lst[2]), float(lst[6]), float(lst[5]), lst[-1]]])]

## Download info

In [5]:
# --- 0. Prep depth, latitude, and longitude lists for Moho interpolation ---
dplst = []
latlst = []
lonlst = []

# --- 1. Get station info
# Set time window for station search (1 hour period starting from 2010-01-01)
starttime = UTCDateTime('2010-01-01T00:00:00')
endtime = starttime + 3600

# Define geographic search parameters
station_search_lower_range=0  # Minimum search radius (0 degrees)
station_search_upper_range=1.0  # Maximum search radius (1 degree from target)

# Target coordinates for station search
search_lo = 69.043
search_la = 34.541

# Query IRIS database for station information; retrieves metadata and instrument response for stations matching criteria
inventory = client.get_stations(level='response',  # Get instrument response data
                               channel='BH*',      # Broadband horizontal channels
                               network="IU",       
                               location = '',
                               starttime=starttime,  # Start of time window
                               endtime=endtime,      # End of time window
                               latitude=search_la,   # Center latitude
                               longitude=search_lo,  # Center longitude
                               minradius=station_search_lower_range,  # Min search radius
                               maxradius=station_search_upper_range)  # Max search radius

# Display the found station information
print(inventory)

# Extract specific station metadata for later use
net = inventory[0]                # Network object (contains network-level metadata)
stn = inventory[0][0]             # Station object (contains station-level metadata)
stla = inventory[0][0].latitude   # Station latitude
stlo = inventory[0][0].longitude  # Station longitude

# --- 2. Find the corners for a study region
# Define anchor point (point1), rectangle dimensions, other parameters
anchor = (78.2, 30.9)  # Starting point
lat1 = anchor[1] * (math.pi / 180)  # Anchor latitude in radians
lon1 = anchor[0] * (math.pi / 180)  # Anchor longitude in radians
wd = 780  # Width of rectangle in kilometers
lt = 380  # Length of rectangle in kilometers
brng = 28*np.pi/180 # Bearing is 28 degrees converted to radians
R = 6378.1 # Radius of the Earth

# Calculate second corner point (point2): moving 'wd' km east from anchor point
lat2 = math.asin( math.sin(lat1)*math.cos(wd/R) +
             math.cos(lat1)*math.sin(wd/R)*math.cos(brng))
lon2 = lon1 + math.atan2(math.sin(brng)*math.sin(wd/R)*math.cos(lat1),
                     math.cos(wd/R)-math.sin(lat1)*math.sin(lat2))
point2 = (lon2*180/np.pi, lat2*180/np.pi)  # Convert back to degrees

# Calculate third corner point (point3) - moving 'lt' km north from anchor
lat3 = math.asin( math.sin(lat1)*math.cos(lt/R) +
             math.cos(lat1)*math.sin(lt/R)*math.cos(brng-np.pi/2))  # Subtract 90° (π/2) to go north
lon3 = lon1 + math.atan2(math.sin(brng-np.pi/2)*math.sin(lt/R)*math.cos(lat1),
                     math.cos(lt/R)-math.sin(lat1)*math.sin(lat2))
point3 = (lon3*180/np.pi, lat3*180/np.pi)  # Convert back to degrees

# Calculate fourth corner point (point4) - moving 'wd' km east from point3
lat1 = point3[1] * (math.pi / 180) # Reset lat1/lon1 to point3 coordinates
lon1 = point3[0] * (math.pi / 180)
lat4 = math.asin( math.sin(lat1)*math.cos(wd/R) +
             math.cos(lat1)*math.sin(wd/R)*math.cos(brng))
lon4 = lon1 + math.atan2(math.sin(brng)*math.sin(wd/R)*math.cos(lat1),
                     math.cos(wd/R)-math.sin(lat1)*math.sin(lat2))
point4 = (lon4*180/np.pi, lat4*180/np.pi)  # Convert back to degrees

# Form a path to select the earthquakes
p = Path([anchor, point2, point4, point3])
print(p)

# --- 3. Select and process the earthquakes
mag_min = 3  # Minimum magnitude threshold for earthquake selection

# Open output files for writing selected earthquake data
# fw1 = open('selected.txt','w')        # File for earthquakes with station info
# fw2 = open('selected_for_search.txt','w')  # File with timestamps for data search
fw3 = open(name,'w')     # File with all regional earthquake info

# Loop through each earthquake in the catalog
for i in range(0,len(cata)):
    # Extract earthquake parameters from catalog
    evla = float(cata[i,1])  # Event latitude
    evlo = float(cata[i,2])  # Event longitude  
    evdp = float(cata[i,3])  # Event depth
    mag = float(cata[i,4])   # Event magnitude
    dperr = float(cata[i,5]) # Depth error
    dpmin = float(cata[i,6]) # Minimum depth
    star = cata[i,7] # Star flag
    star = star[0]
    if star == '*':
        star = '*'
    else:
        star = '-'
    
    # Check if earthquake is within study region AND meets magnitude threshold
    if p.contains_points([(evlo,evla)]) and mag >= mag_min:
        
        # # Write earthquake info to output files
        # if cata[i,-1] != '--':  # If station info exists
        #     fw1.write(str(UTCDateTime(cata[i,0]))+' '+str(evla)+' '+str(evlo) + '   ' +str(evdp)+'   '+ cata[i,-1]+'\n')
        #     fw2.write('\''+str(UTCDateTime(cata[i,0]))+'\',\n')
        fw3.write(str(UTCDateTime(cata[i,0]))+'  '+str(evla)+'  '+str(evlo) + '  ' +str(evdp)+'  '+ str(mag)+'  '+ str(dperr)+'  ' + str(star)+'\n')
        dplst.append(evdp)
        latlst.append(evla)
        lonlst.append(evlo)
        
        # Set up time window for data download
        starttime = UTCDateTime(cata[i,0])
        endtime = UTCDateTime(cata[i,0]) + 3600 
        origin_time = starttime
        
        # Extract time components for filename construction
        yr = origin_time.year
        jd = origin_time.julday
        hr = origin_time.hour
        mins = origin_time.minute
        sec = origin_time.second
        msec = origin_time.microsecond
        
        # Calculate distance and azimuth from station to earthquake
        geo = gps2dist_azimuth(stla, stlo, evla, evlo)
        epi_dist = geo[0] / 1000  # Epicentral distance in km
        if epi_dist < 250:  # Skip if too close to station
            print('!!WARNING: Too close to station ' + cata[i,0].split("T")[0])
            continue
        baz = geo[1]  # Back azimuth
        gcarc = kilometer2degrees(epi_dist)  # Convert distance to degrees
        
        # Calculate P-wave arrival time using travel time model
        arrival = model.get_travel_times(source_depth_in_km=evdp, distance_in_degree=gcarc,
                                         phase_list=['p','P'])
        P_arrival_time_at_stn = origin_time + arrival[0].time  # Add travel time to origin time

        # Define time window around P arrival for data download
        seconds_before_P = P_arrival_time_at_stn - 60   # 60 seconds before P arrival
        seconds_after_P = P_arrival_time_at_stn + 400   # 400 seconds after P arrival
        
        # Print event information
        print('P Arrival time for event ' + cata[i,0].split("T")[0] + ' M' + str(mag)+' is: ' + str(P_arrival_time_at_stn))
        print('Origin time for this event: ' + str(arrival[0].time))
        print('Event latitude: ' + str(evla) + '  Event longitude: ' + str(evlo))
        
        # Download waveform data from IRIS
        try:
            stream = client.get_waveforms(network = net.code, station = stn.code, location = '', channel = 'BH*', starttime = seconds_before_P, endtime = seconds_after_P, attach_response=True)
        except:
            print('!!WARNING: No data for event ' + cata[i,0].split("T")[0])
            continue
            
        # Check data quality
        number_of_traces = len(stream)
        number_of_locs = int(len(stream) / 3)
        print(str(number_of_traces) + ' traces found for event ' + cata[i,0].split("T")[0])
        if number_of_traces > 3:  # Skip if data is fragmented
            print('!!WARNING: Fragmented data')
            continue
            
        # Rename (and rotate if uncomment)
        stream[0].stats.channel = 'BHN'  # Rename channels, works for LSA in particular
        stream[1].stats.channel = 'BHE'
        stream[2].stats.channel = 'BHZ'
        # result = gps2dist_azimuth(stla, stlo, evla, evlo, a=6378137.0, f=0.0033528106647474805)
        # a0 = max(stream[1].data)  # Store original amplitude for comparison
        # stream.rotate(method = 'NE->RT',back_azimuth = result[1])
        
        # # Check if rotation was successful
        # if abs(max(stream[1].data) - a0)/a0 < 0.01:
        #     print('!!WARNING: Traces might not be rotated')
            
        # Write raw data to SAC files
        for tr in stream:
            loc = tr.stats.location
            chan = tr.stats.channel

            # Construct SAC filename with event and station info
            date_str = str(origin_time)
            sacnm = raw_directory + '/' + net.code + '.' + stn.code + '.' + str(loc) + '.' + date_str + '.' + chan + '.SAC'
            print('(Raw) Event written to SAC file: ' + sacnm)
            
            # Convert ObsPy trace to SAC format
            sac = SACTrace.from_obspy_trace(tr)
            sac.kcmpnm = chan      # Component name
            sac.gcarc = gcarc      # Great circle distance
            sac.baz = baz          # Back azimuth
            sac.evlo = evlo        # Event longitude
            sac.evla = evla        # Event latitude
            sac.stlo = stlo        # Station longitude
            sac.stla = stla        # Station latitude
            sac.evdp = evdp        # Event depth
            sac.user0 = dperr      # Depth error
            sac.user1 = dpmin      # Minimum depth
            
            # Assign catalog source code to user3 field
            if cata[i,-1] == '*Alvizuri': sac.user3 = 1
            elif cata[i,-1] == '*Craig': sac.user3 = 2
            elif cata[i,-1] == '*Baur': sac.user3 = 3
            elif cata[i,-1] == '*Parija': sac.user3 = 4
            elif cata[i,-1] == '*Michailos': sac.user3 = 5
            elif cata[i,-1] == '*GANSSER': sac.user3 = 6
            elif cata[i,-1] == '*Monsalve': sac.user3 = 7
            elif cata[i,-1] == '*Jiang': sac.user3 = 8
            elif cata[i,-1] == '*gmt': sac.user3 = 9
            elif cata[i,-1] == '*isc': sac.user3 = 10
            else: sac.user3 = 0
            
            sac.mag = mag          # Magnitude
            sac.a = P_arrival_time_at_stn  # P arrival time
            sac.o = origin_time    # Origin time
            sac.write(sacnm)       # Write SAC file

print('Done!')
# fw1.close()
# fw2.close()
fw3.close()

Inventory created at 2025-08-18T22:59:59.158600Z
	Created by: IRIS WEB SERVICE: fdsnws-station | version: 1.1.52
		    http://service.iris.edu/fdsnws/station/1/query?starttime=2010-01-01...
	Sending institution: IRIS-DMC (IRIS-DMC)
	Contains:
		Networks (1):
			IU
		Stations (1):
			IU.KBL (Kabul, Afghanistan)
		Channels (3):
			IU.KBL..BHZ, IU.KBL..BHN, IU.KBL..BHE
Path(array([[78.2       , 30.9       ],
       [82.31343758, 37.02244506],
       [78.58307354, 38.57196077],
       [74.45398787, 32.45360227]]), None)
P Arrival time for event 2022-01-04 M4.3 is: 2022-01-04T17:17:19.935506Z
Origin time for this event: 87.60450573425688
Event latitude: 33.0339  Event longitude: 76.074
3 traces found for event 2022-01-04
(Raw) Event written to SAC file: /Users/emmanuelzheng/Downloads/Synthetics/Seiobs/WT_Raw_Data//IU.KBL..2022-01-04T17:15:52.331000Z.BHN.SAC
(Raw) Event written to SAC file: /Users/emmanuelzheng/Downloads/Synthetics/Seiobs/WT_Raw_Data//IU.KBL..2022-01-04T17:15:52.331000Z.BHE.

## Append d-H values

In [6]:
# Open the Moho profile text file; read all lines from the file into a list of strings
fr1 = open('/Users/emmanuelzheng/Downloads/Synthetics/Xiaohan/Moho_RF_int.txt','r')
lines_r1 = fr1.readlines()

# Define the longitude grid from 74° to 110° with 145 points (0.25° spacing)
xmo = np.linspace(74, 110, 145)
# Define the latitude grid from 26° to 42° with 65 points (0.25° spacing)
ymo = np.linspace(26, 42, 65)
# Create 2D mesh grid arrays for the longitude and latitude coordinates
[xm, ym] = np.meshgrid(xmo,ymo)
# Initialize the Moho depth grid with zeros (shape 145 x 65, matching xmo x ymo)
zmo = np.zeros([145, 65])

# Loop over each line from the Moho file to populate the zmo grid
i = 0
for line in lines_r1:
    # Split the current line into whitespace-separated fields
    elements = line.split()
    # If the depth value (third column) is valid (< 10000), assign it into zmo
    if float(elements[2]) < 10000:
        zmo[i%145, i//145] = float(elements[2])
    # Otherwise, set a default/fallback value (e.g., 40 km) for missing/invalid entries
    else:
        zmo[i%145, i//145] = 40 #np.nan
    # Increment the flat index counter for the next line
    i = i + 1
# Transpose the Moho depth grid so its shape aligns with [len(ymo), len(xmo)]
zmo = zmo.transpose()

# Make a working copy of depth list (dplst) so original is not modified
d_h = dplst.copy()
# Build a 2D linear interpolator over the Moho grid (longitude xmo, latitude ymo, depth zmo)
f = interpolate.interp2d(xmo, ymo, zmo, kind='linear')

# Loop over all points for which we want to compute adjusted depths
for i in range(len(dplst)):
    # Compute nearest-grid indices for current point's lon/lat relative to xmo/ymo spacing
    ix = round((lonlst[i] - np.min(xmo))/0.25)
    iy = round((latlst[i] - np.min(ymo))/0.25)
    # Option A: subtract grid node depth (nearest-neighbor) — kept for reference
    #d-H[i] = dplst[i] - zmo[iy, ix]
    # Option B: subtract bilinearly interpolated Moho depth at the exact lon/lat
    d_h[i] = dplst[i] - f(lonlst[i], latlst[i])

# Path to the file you previously wrote (update as needed)
file_path = name
# Read existing rows
with open(file_path, "r") as f:
	rows = [ln.rstrip("\n") for ln in f if ln.strip()]
# Overwrite file with appended d_h column
with open(file_path, "w") as f:
	for i, row in enumerate(rows):
		f.write(f"{row}  {float(d_h[i])}\n")

/var/folders/1b/q38gzq296kvdfg7pyzwtv9q80000gn/T/ipykernel_4633/2739738522.py:33: DeprecationWarning: `interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.14.0.

For legacy code, nearly bug-for-bug compatible replacements are
`RectBivariateSpline` on regular grids, and `bisplrep`/`bisplev` for
scattered 2D data.

In new code, for regular grids use `RegularGridInterpolator` instead.
For scattered data, prefer `LinearNDInterpolator` or
`CloughTocher2DInterpolator`.

For more details see
`https://scipy.github.io/devdocs/notebooks/interp_transition_guide.html`

  f = interpolate.interp2d(xmo, ymo, zmo, kind='linear')
/var/folders/1b/q38gzq296kvdfg7pyzwtv9q80000gn/T/ipykernel_4633/2739738522.py:43: DeprecationWarning: `interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.14.0.

For legacy code, nearly bug-for-bug compatible replacements are
`RectBivariateSpline` on regular grids, and `bisplrep`/`bisplev` for
scattered 2D data.

In new code, for regular grids